# Loading the pre-processed data

In [1]:
import pandas as pd
# Load the preprocessed dataset
df = pd.read_csv(r"C:\Users\rajak\Downloads\AI Internship\clinical_notes_preprocessed_no_spellcheck.csv")
# Display basic information
print(df.shape)
df.head()

(158114, 3)


,patient_id,task,note_preprocessed
0,0,Paraphrasing,discharge summary patient year old male modera...
1,1,Coreference Resolution,discharge summary admission date insert date d...
2,2,Summarization,hospital course summary admission date insert ...
3,3,Relation Extraction,discharge summary patient year old male hospit...
4,4,Relation Extraction,discharge summary patient information name red...


In this case we loads the existing preprocessed data and printed his for the visualisation of the data.

# Raw pre-processed text

In [4]:
# Extract preprocessed clinical notes
texts = df["note_preprocessed"].astype(str).tolist()
# Display one sample text
print("Sample preprocessed clinical note:\n")
print(texts[0])

Sample preprocessed clinical note:

discharge summary patient year old male moderate acute respiratory distress syndrome covid hospital course patient admitted hospital symptom fever dry cough dyspnea physical therapy acute ward patient experienced coughing attack induced oxygen desaturation dyspnea change position deep breathing avoid rapid deterioration respiratory failure step step approach used position change breathing exercise adapted avoid prolonged coughing oxygen desaturation close monitoring patient managed perform strength walking exercise low level exercise progression low initially increased daily hospital discharge rehabilitation clinic day clinical outcome patient discharged day rehabilitation clinic making satisfactory progress symptom resolved follow patient receive follow care rehabilitation clinic regular monitoring progress rehabilitation exercise full recovery new symptom concern reported clinic immediately overall impression patient responded well treatment approp

displayed the pre-processed data. That needs to be embeded.

# clinical text to vector- TF-IDF

In [6]:
pip install sentence-transformers

   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.3 MB 7.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.3 MB 8.4 MB/s eta 0:00:01
   ------------------- -------------------- 5.0/10.3 MB 9.1 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.3 MB 9.1 MB/s eta 0:00:01
   --------------------------------- ------ 8.7/10.3 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  10.2/10.3 MB 9.0 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.3 MB 8.4 MB/s  0:00:01
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   -------------------------------------- - 2.6/2.7 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 9.3 MB/s  0:00:00
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   - -------------------------------------- 3.9/113.8 MB 29.4 MB/s eta 0:00:04
   --- ----------------

downloading the essential libraries for converting the clinical text to the vector.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2)
)

using TF-IDF (Basic model) to convert the text terms to the vectors.

In [5]:
tfidf_vectors = vectorizer.fit_transform(texts)
print("TF-IDF matrix shape:", tfidf_vectors.shape)

TF-IDF matrix shape: (158114, 3000)


Converted the text format to the vector format for the further comptation

Displaying a sample vector

In [6]:
sample_index = 4
print("Sample preprocessed note:\n")
print(texts[sample_index])
print("\nCorresponding FULL TF-IDF vector:\n")
print(tfidf_vectors[sample_index].toarray()[0][:60])

Sample preprocessed note:

discharge summary patient information name redacted age gender male admission date redacted discharge date n medical record number redacted hospital course summary patient admitted icu symptom dyspnea heavy dry cough fever after testing positive covid despite initial ability exercise sit chair physical therapist patient respiratory condition progressively worsened requiring intubation proning patient large amount bronchial mucus respiratory failure necessitated regular suctioning respiratory therapy manual airway clearance technique employed physical therapist increase effective airway clearance avoiding alveolar collapse success after extubation patient continued require intensive manual airway clearance technique nasal rinsing airway suctioning additional physical therapy intervention including passive range motion assisted exercising mobilization also utilized time writing patient remained icu without ventilatory support summary event patient presented sym

clincal pre-pocessed tecxt is converted into the numerical vectors using TF-IDF vectorisation. This approch provides an interpretable method for representing clinical text numerically, which enables similarity analysis and further analysis.

replace the pre-processed data to the new vector converted data

In [7]:
import numpy as np
# Convert sparse TF-IDF matrix to dense array
tfidf_dense = tfidf_vectors.toarray()
# Replace the preprocessed text column with vectorized data
df["note_preprocessed"] = list(tfidf_dense)
# Display the first few rows
df.head()

,patient_id,task,note_preprocessed
0,0,Paraphrasing,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,Coreference Resolution,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,Summarization,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,Relation Extraction,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,Relation Extraction,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Visualisation of the vector converted data.

# Cosine similarity

Finding cosine similarity among first 5 patients

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Select first 5 samples from TF-IDF matrix
tfidf_first5 = tfidf_vectors[:5]

# Compute cosine similarity among first 5 samples
similarity_first5 = cosine_similarity(tfidf_first5)

# Convert to DataFrame for readability
similarity_df = pd.DataFrame(
    similarity_first5,
    index=[f"Sample_{i}" for i in range(5)],
    columns=[f"Sample_{i}" for i in range(5)]
)

# Display result
similarity_df


,Sample_0,Sample_1,Sample_2,Sample_3,Sample_4
Sample_0,1.000000,0.321947,0.312308,0.293158,0.214100
Sample_1,0.321947,1.000000,0.427197,0.129636,0.230753
Sample_2,0.312308,0.427197,1.000000,0.223862,0.220615
Sample_3,0.293158,0.129636,0.223862,1.000000,0.219358
Sample_4,0.214100,0.230753,0.220615,0.219358,1.000000


Calculating the cosine similarity for the first 50 subjects

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# Select first 50 TF-IDF vectors
tfidf_first50 = tfidf_vectors[:50]
# Compute cosine similarity
similarity_first50 = cosine_similarity(tfidf_first50)
# Create labels (use patient_id if available)
if "patient_id" in df.columns:
    labels = df["patient_id"].iloc[:50].tolist()
else:
    labels = [f"Patient_{i}" for i in range(50)]
# Convert to DataFrame for readability
similarity_df_50 = pd.DataFrame(
    similarity_first50,
    index=labels,
    columns=labels
)
# Display the similarity matrix
similarity_df_50.head()

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,1.000000,0.321947,0.312308,0.293158,0.214100,0.380288,0.479607,0.233067,0.267236,0.287114,...,0.085118,0.110828,0.036073,0.048946,0.131223,0.092087,0.113414,0.026222,0.050048,0.080726
1,0.321947,1.000000,0.427197,0.129636,0.230753,0.193097,0.218754,0.214454,0.164848,0.214025,...,0.106563,0.132012,0.049722,0.052332,0.142692,0.066701,0.134207,0.041769,0.237683,0.127448
2,0.312308,0.427197,1.000000,0.223862,0.220615,0.191498,0.260992,0.252983,0.219756,0.249463,...,0.120295,0.119230,0.053207,0.061623,0.153833,0.095674,0.158061,0.058053,0.195471,0.114978
3,0.293158,0.129636,0.223862,1.000000,0.219358,0.356663,0.412893,0.313689,0.295351,0.335993,...,0.102114,0.098555,0.088448,0.084450,0.121035,0.079232,0.120031,0.071932,0.080776,0.096941
4,0.214100,0.230753,0.220615,0.219358,1.000000,0.186969,0.162246,0.224942,0.302241,0.176915,...,0.088956,0.072551,0.068527,0.050535,0.092658,0.060154,0.134919,0.045948,0.049163,0.093905


Has the dataset has 1,50,000 dataset. I have selecting the first 1000 patients alone for the further processing 

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Number of patients to consider
N = 500

# Select first 500 TF-IDF vectors (sparse, memory-safe)
tfidf_first500 = tfidf_vectors[:N]

# Compute cosine similarity
similarity_first500 = cosine_similarity(tfidf_first500)

# Create labels using patient_id if available
if "patient_id" in df.columns:
    labels = df["patient_id"].iloc[:N].tolist()
else:
    labels = [f"Patient_{i}" for i in range(N)]

# Convert similarity matrix to DataFrame
similarity_df_500 = pd.DataFrame(
    similarity_first500,
    index=labels,
    columns=labels
)

# Display the first few rows
similarity_df_500.head()


,0,1,2,3,4,5,6,7,8,9,...,520,521,522,523,524,525,526,527,528,529
0,1.000000,0.321947,0.312308,0.293158,0.214100,0.380288,0.479607,0.233067,0.267236,0.287114,...,0.029592,0.056992,0.137948,0.052150,0.207042,0.086784,0.050787,0.102172,0.053731,0.053836
1,0.321947,1.000000,0.427197,0.129636,0.230753,0.193097,0.218754,0.214454,0.164848,0.214025,...,0.040667,0.078728,0.143239,0.057569,0.127451,0.069234,0.059463,0.052470,0.062030,0.057334
2,0.312308,0.427197,1.000000,0.223862,0.220615,0.191498,0.260992,0.252983,0.219756,0.249463,...,0.052478,0.093864,0.140140,0.059721,0.162284,0.061977,0.066285,0.075420,0.083192,0.060283
3,0.293158,0.129636,0.223862,1.000000,0.219358,0.356663,0.412893,0.313689,0.295351,0.335993,...,0.028466,0.072641,0.115585,0.049525,0.147369,0.070163,0.070125,0.080786,0.097333,0.079922
4,0.214100,0.230753,0.220615,0.219358,1.000000,0.186969,0.162246,0.224942,0.302241,0.176915,...,0.034572,0.093497,0.093960,0.042834,0.158568,0.038183,0.052406,0.058049,0.058463,0.055640


Just printing head of the sample distance that have been calculated from the first 500 subjects

# Pre-processed to vectorised csv

creating the single unified code that process everything from the pre-processed data to vectorised csv